Football Injury Datasets

https://www.kaggle.com/competitions/nfl-playing-surface-analytics/data

https://www.kaggle.com/datasets/smadler92/nfl-pfr

https://nflreadr.nflverse.com/articles/dictionary_injuries.html

https://github.com/ali-ce/datasets/blob/master/NFL/Concussion%20Injuries%202012-2014.csv

https://github.com/sammieerne/NFL-Injury-Analysis